In [58]:
import json
import nltk
import tensorflow
import random
import numpy as np
from tensorflow.keras.preprocessing import sequence
import pickle

In [43]:
intents= json.loads(open(r"../datasets/chatbot/intents.json").read())

In [44]:
lemmatizer= nltk.stem.WordNetLemmatizer()

In [45]:
words = [] 
classes = [] 
documents_x = [] 
documents_y= []
ignore_letters = ["?", "!", ".", ","] 

In [46]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)

        documents_x.append(word_list)
        documents_y.append(intent['tag'])

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [47]:
nltk.download('wordnet')
words = [lemmatizer.lemmatize(word.lower())
         for word in words if word not in ignore_letters]
words = sorted(list(set(words)))
classes = sorted(list(classes))
word_index = {word: index for index, word in enumerate(words)}

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Rohan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
training = []
output = []
output_empty = [0 for _ in range(len(classes))]

In [50]:
for x, document in enumerate(documents_x):
    bag = []
    word_indices = [word_index.get(lemmatizer.lemmatize(word.lower()), len(word_index)) for word in document]
    output_row = output_empty[:]
    output_row[classes.index(documents_y[x])] = 1

    training.append(word_indices)
    output.append(output_row)

In [52]:
max_length = max(len(doc) for doc in training)
training = pad_sequences(training, maxlen=max_length, padding='post')
output = np.array(output)

In [53]:
vocab_size = len(word_index) + 1
model = tensorflow.keras.models.Sequential()
model.add(tensorflow.keras.layers.Embedding(vocab_size, 128, input_length=max_length))
model.add(tensorflow.keras.layers.GlobalAveragePooling1D())
model.add(tensorflow.keras.layers.Dense(64, activation='relu'))
model.add(tensorflow.keras.layers.Dropout(0.5))
model.add(tensorflow.keras.layers.Dense(len(output[0]), activation='softmax'))

In [54]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(training, output, epochs=200, batch_size=5)

Epoch 1/200


8/8 [==============================] - 1s 8ms/step - loss: 1.3751 - accuracy: 0.2250
Epoch 2/200
8/8 [==============================] - 0s 9ms/step - loss: 1.3614 - accuracy: 0.3500
Epoch 3/200
8/8 [==============================] - 0s 9ms/step - loss: 1.3343 - accuracy: 0.4750
Epoch 4/200
8/8 [==============================] - 0s 9ms/step - loss: 1.3200 - accuracy: 0.5000
Epoch 5/200
8/8 [==============================] - 0s 10ms/step - loss: 1.2976 - accuracy: 0.5500
Epoch 6/200
8/8 [==============================] - 0s 11ms/step - loss: 1.2670 - accuracy: 0.6250
Epoch 7/200
8/8 [==============================] - 0s 10ms/step - loss: 1.2410 - accuracy: 0.7500
Epoch 8/200
8/8 [==============================] - 0s 12ms/step - loss: 1.2074 - accuracy: 0.6750
Epoch 9/200
8/8 [==============================] - 0s 12ms/step - loss: 1.1548 - accuracy: 0.7250
Epoch 10/200
8/8 [==============================] - 0s 12ms/step - loss: 1.1030 - accuracy: 0.7500
Epoch 11/200
8/8 [=================

In [55]:
model.save('chatbot_model.h5')

In [56]:
with open('data.pickle', 'wb') as f:
    pickle.dump((words, classes, training, output), f)

In [59]:
def bag_of_words(s, word_index, max_length):
    word_indices = [word_index.get(lemmatizer.lemmatize(word.lower()), len(word_index)) for word in s]
    padded_indices = pad_sequences([word_indices], maxlen=max_length, padding='post')[0]
    return padded_indices

In [60]:
def chat(prompt: str):
    print("Start talking with the bot (type quit to stop)!")

    input_vector = bag_of_words(nltk.word_tokenize(prompt), word_index, max_length)
    results = model.predict(np.array([input_vector]))[0]
    results_index = np.argmax(results)
    tag = classes[results_index]

    for tg in intents['intents']:
        if tg['tag'] == tag:
            responses = tg['responses']
    print(random.choice(responses))

In [63]:
chat("hiiiiiiiiiiiiiii")

Start talking with the bot (type quit to stop)!
1/1 [==============================] - 0s 45ms/step
Hi there! I'm your personal health assistant. What would you like to discuss?
